In [4]:
from time import sleep
import chardet
from github3 import login
from pycurl import Curl

## GitHub API attributes

In [11]:
api_rate_limit_per_hour = 5000
sec_between_requests = 60 * 60 / api_rate_limit_per_hour
url_repos = "https://api.github.com/repos"

## Utility functions for GitHub API

In [7]:
def gh_userpwd(gh_username, gh_oauth_key):
    """ Returns string version of GitHub credentials to be passed to GitHub API
    
    Args:
        gh_username: GitHub username for GitHub API
        gh_oauth_key: (String) GitHub oauth key
    """
    return('%s:%s' %(gh_username, gh_oauth_key))

def sleep_gh_rate_limit():
    """ Sleep for an amount of time that, if done between GitHub API requests for a full hour,
    will ensure the API rate limit is not exceeded.
    """    
    sleep(sec_between_requests + 0.05) 
    
def add_page_num(url, page_num):
    """Add page number to GitHub API request"""
    if "?" in url:
        return "%s&page=%s" %(url, page_num)
    else:
        return "%s?page=%s" %(url, page_num)
    
def validate_response_found(parsed, message = ""):
    """ Check that the GitHub API returned a valid response
    Raises ValueError if response was not found
    
    Args:
        parsed: Parsed JSON response as a dict
        message: Extra info to print
    """
    if "message" in parsed:
        if parsed["message"] == "Not Found":
            raise ValueError("Parsed response has message: Not Found. Further information:\n%s" %message)

def gh_curl_response(url, gh_username, gh_oauth_key):
    """
    Returns the parsed curl response from the GitHub API
    Combines pages if applicable
    
    params:
        url: URL e.g. 'https://api.github.com/repos/samtools/samtools'
        gh_username: GitHub username for GitHub API
        gh_oauth_key: (String) GitHub oauth key
        
    returns:
        Parsed API response. Returns a list of dicts, one for each record, or just one
        dict if the response was a single dict.
        
    """
    page_num = 1
    results = []
    prev_response = None
    while True:
        buffer = BytesIO()
        c = pycurl.Curl()
        c.setopt(c.URL, add_page_num(url, page_num))
        c.setopt(c.USERPWD, gh_userpwd(gh_username, gh_oauth_key))
        c.setopt(c.WRITEDATA, buffer)
        sleep_gh_rate_limit()
        try:
            c.perform()
        except pycurl.error as e:
            print(url)
            raise e
        c.close()
        body = buffer.getvalue()
        try:
            parsed = json.loads(body.decode())
            if "message" in parsed:
                if "API rate limit exceeded" in parsed["message"]:
                    raise PermissionError(parsed["message"])
        except JSONDecodeError:
            print("Caught JSONDecodeError. Returning empty list for URL %s" % url)
            return []
        validate_response_found(parsed, add_page_num(url, page_num))
        if type(parsed) is dict:
            return parsed
        else:
            if len(parsed) == 0:
                break
            else:
                if parsed == prev_response:
                    # Sometimes GitHub API will return the same response for any provided page num
                    break
                else:
                    prev_response = parsed
                    results = results + parsed
                    page_num = page_num + 1
    return results